## Adding a new quiz to Coder Quiz

These steps will talk you through everything you need to know in order to create a new quiz for Coder Quiz. Specifically we will cover -

1. Writing the questions for the quiz

2. Using existing validators and creating new validators

3. Making the quiz accessable to a user

4. Optionally customising the default CSS and HTML of the page the quiz renders on

5. How to create a database record to store the results from users

Note that a lot of these steps have code that is already handled and written by Coder Quiz, so there is minimal input needed from the developer to use them.

I recommend having the base Coder Quiz repository in front of you and coding as you go. Check out a new branch to work on, or if you'd like to see the end results of this guide - checkout the branch `example_quiz`

## Step 1: Writing the questions for the quiz

### The questions

Let's keep things fairly simple, and just have two questions, a Python file upload, and an image upload. This will cover the broad range of what Coder Quiz can handle, but we'll point out what other options are available at appropriate times too.

Lets say we want a quiz like this

```
Q1: Enter the third month of the year
Q2: Enter a number between 200 and 400 (inclusive)
Q3: Upload a Python file with a function of your choice in it
Q4: Upload an image of a wombat
```

### Adding it to Coder Quiz

1. Create a new form - Coder Quiz has three .py files where it stores forms - `forms_scie2100.py`, `forms_biol3014.py`, and `forms.py` . The first two are specifically for the two University of Queensland Bioinformatics courses, and the second is a general forms file that stores items such as the Practice Quiz, the Signup form, and the forms we use to query student's grades. 
    
    To keep things simple lets start a whole new file - `forms_example.py`
    
    This will mean we have to import a few things, that we won't have to if we're adding a new form to an existing file.
    
    Add the following imports -
    ```
    from flask_wtf import FlaskForm
    from wtforms import StringField, SubmitField, FileField
    from wtforms.validators import DataRequired
    from form_validators import CorrectAnswer, CheckNumberRange
    ```
    
    Coder Quiz is written in the Python webframework Flask, and we use the WTForms library to create the interactive forms. flask_wtf is a module that lets you use WTForms in Flask. Flask is by design like this - pretty bare bones, and with lots of optional modules you can add onto it.
    
    The *first import line* imports the FlaskForm object, which are what our forms are going to be, the *second import line* imports some field types that we'll use when building our form, and the *third import line* imports some pre-existing validators for our forms. Not that these are all external modules to Coder Quiz, i.e. if you're interested there are a lot more types of fields, and default validators that you could utilise. The great thing about the validators is that we can write custom validators for our form, which is where a lot of the power in Coder Quiz comes from. This is what the *fourth import line* is importing - custom validators, including two that exist already - `CorrectAnswer` and `CheckNumberRange`
    
    Now add the following code to create our new form -
    

    
    
    

    

```python
class ExampleQuiz(FlaskForm):
    form_name = "ExampleQuiz"

    q1 = StringField("Question 1: Enter the third month of the year",
                      validators=[CorrectAnswer("March"),
                                  DataRequired( "You should supply an answer to each question")],
                      filters=[lambda v: None if v == '' else v])

    q2 = StringField("Question 2: Enter a number between 200 and 400 (inclusive)",
                      validators=[CheckNumberRange(200, 400), DataRequired(
                          "You should supply an answer to each question")],
                      filters=[lambda v: None if v == '' else v])

    q3_code = FileField(
        'Question 3: Upload a Python file with a function of your choice in it',
        validators=[DataRequired("Please attach your code for Question 3.")],
        filters=[lambda v: None if v == '' else v])

    q4_image = FileField(
        "Question 4: Upload an image of a wombat",
        validators=[DataRequired("Please attach an image")],
        filters=[lambda v: None if v == '' else v])

    check = SubmitField("Check  answers")

    submit = SubmitField("Submit answers")

    submission_form = "SubmissionExampleQuiz"

    questions = ['q1', 'q2', 'q3_code', 'q4_image']
```

**Required features of a new quiz** 

For each form object you create, you should add the following fields -

1. `form_name` - If this is an In Class assessment put the InClass in the name, as Coder Quiz uses this to style these slightly differently
2. `q1`, `q2`, `q3_code`, etc... - These can be named anything, except make sure to use `_code` and `_image` for those fields as the validation handlers will treat these fields differently 
3. `check = SubmitField("Check  answers")` - Creates a button that will let the user check their answers
4. `submit = SubmitField("Submit answers")` - Creates a button that will let the user submit their answers
5. `submission_form = "SubmissionExample"` - Tells Coder Quiz which submission form to associate this form with
6. `questions` - A list of all the question names. Just a quick and lazy way of being able to iterate through all the fields for a form

Now let's look a little more closely at how we structure one of our questions in our form

```python
    q1 = StringField("Question 1: Enter the third month of the year",
                      validators=[CorrectAnswer("March"),
                                  DataRequired( "You should supply an answer to each question")],
                      filters=[lambda v: None if v == '' else v])
    ```
    
We create a `StringField` object (this just tells the form what kind of field we want to use for this question. Again, this is a WTForms field, so we can use any of the fields they make available, but for this guide we'll stick to `StringField` and `FileField` (and the SubmitField we use to create the buttons). Coder Quiz currently also uses `TextAreaField` (for entering large blocks of text) and `SelectField` (for dropdown select boxes)

These fields take a string, here equal to 

```python 
"Question 1: Enter the third month of the year"
```

Which is just what we'll display to the user when the form is rendered

And a list of validators we want to apply to this field

```python
validators=[CorrectAnswer("March"), DataRequired( "You should supply an answer to each question")]
```

We'll talk more about validators in **Step 2**, but for now we can easily see that in this example we use two validators, `CorrectAnswer()` where we pass in the correct answer, and `DataRequired()` where we pass in a message to display to the user if they haven't supplied an answer to this question when they hit the `Check answers` button

Finally we also supply a list called filters -

```python
filters=[lambda v: None if v == '' else v]
```

This is just a bit of logic that sets the answer that WTForms will validate against to `None` if the response has been left empty.

This is the way each form within Coder Quiz is set out, have a look at some of the more detailed forms within `forms_scie2100.py` or `forms_biol3014.py` to see some more examples of how these forms are currently used.


**You don't have to validate!**

Note that currently Coder Quiz doesn't do any validation of uploaded code (except to check that it ends in .py but this check occurs at a later stage). Because of this, if you look at the list of validators for the code or image upload fields you'll see that we just don't pass in `DataRequired` as a validator.

## Step 2: Using existing validators and creating new validators

External documentation is here https://wtforms.readthedocs.io/en/stable/validators.html but we'll cover the main points in a Coder Quiz focussed manner below.

We saw in **Step 1** that we can use validators that already exist (either from `wtforms.validators` such as `DataRequired` or from the Coder Quiz module `form_validators` such as `CorrectAnswer`). Let's look a little more closely at what's happening with the custom validators and how we can create them.

First lets look at `CorrectAnswer`, which is probably the simplest form validator implemented in Coder Quiz, but also very useful.

```python
class CorrectAnswer(object):
    """
    Custom validator for WTForms to check
    if the correct answer was submitted
    """

    def __init__(self, answers):
        if isinstance(answers, str):
            self.answers = [answers]
        else:
            self.answers = answers

    def __call__(self, form, field):
        # List of error messages that are selected by random
        error_messages = ['Sorry, that\'s not the correct answer.',
                          'Try that again...',
                          'Incorrect answer.',
                          'Please check this answer...',
                          'Oops! Try again...',
                          'Nope! Sorry... try again!',
                          'No, not quite... try again!',
                          'Hmmm, not exactly right...']

        num = randrange(0, len(error_messages))
        message = error_messages[num]

        if field.data is not None:
            for answer in self.answers:
                if answer.strip().upper() == field.data.strip().upper():
                    return

        raise ValidationError(message)
```

**The init and call methods**

There are two main private methods that a custom validator has, `__init__` and `__call__` .

`__init__` is used to setup any of the data we'll need for validation - it can take arguments and set them as attributes, so that we can reuse the same validator but pass in the expected answer, or Python objects to generate the expected answer. Note that not all custom validators need an `__init__` method - `CheckPalindrome` for example doesn't need to take any arguments or do any setting up in order to tell if a string is a palindrome, but for the most part it's important to reusability to have an `__init__` method.

`__call__` is the method that will be called when the user either checks or submits the form, and does the actual validation of the field. It has to take the arguments `self, form, field` because it is overriding an existing method, and we can use `field.data` to get access to the value that the user has entered as input to the field.

So, one basic workflow is to pass in a correct answer when constructing the validator, then when the user checks or submits we take their data from `field.data` and check to see if it is equivelant to the expected answer. If it passes whatever test we need it to, we can just return, but if it doesn't validate correctly then we need to raise a `ValidationError()` - and we can pass in whatever error message we want to display to the user on the rendered form. This means that we can easily display different error messages to the user after evalutating their code.

**Putting this all together**

Lets look even more closely at `CorrectAnswer`, just so we can see how it puts all of this together.

```python
    def __init__(self, answers):
        if isinstance(answers, str):
            self.answers = [answers]
        else:
            self.answers = answers
```

The init method takes the variable `answers`, and if it is a `String` object converts it into a `List` object. For reusability this now means we can use the same method to check if a user-entered answer matches a single `String` or if it matches *any* `String` within a list.

```python
    def __call__(self, form, field):
        # List of error messages that are selected by random
        error_messages = ['Sorry, that\'s not the correct answer.',
                          'Try that again...',
                          'Incorrect answer.',
                          'Please check this answer...',
                          'Oops! Try again...',
                          'Nope! Sorry... try again!',
                          'No, not quite... try again!',
                          'Hmmm, not exactly right...']

        num = randrange(0, len(error_messages))
        message = error_messages[num]
```

Here we're just generating and selecting one possible responses to an incorrect answer. These are all just generic messages, but we could also create more specific error messages and return those.

```python
        if field.data is not None:
            for answer in self.answers:
                if answer.strip().upper() == field.data.strip().upper():
                    return

        raise ValidationError(message)
```

Now, provided that the user has entered something into the input field, we check all of the possible answers in the `List` of answers we created in the `__init__` method based on what we passed into `CorrectAnswer` when we created it. In this case, we used ```python CorrectAnswer("March")``` - check the code we wrote for `q1` in the `ExampleQuiz` in `forms_example.py` 

So this means the `List` of correct answers just contains'March', and if the user input currently stored in `field.data` matches that we return from the function. If it was a longer list, we'd return as soon as we made one correct match, and if we don't find a match then we raise a `ValidationError` with the randomly generated error message we created.

As you can see in the code, `CorrectAnswer` also strips any whitespace and converts both `String` objects to uppercase before checking for equality.<span style="color:red"> Make sure if you're implementing new validators that you think carefully about input that could be entered and what should be accepted as a valid answer.</span>


**Comparing to CheckNumberRange()**

Let's make sure this makes sense by looking at how `CheckNumberRange` implements this workflow for validation -

```python
class CheckNumberRange(object):
    def __init__(self, lower, upper, hint=""):
        self.lower = lower
        self.upper = upper
        self.hint = hint

    def __call__(self, form, field):

        if field.data is not None:
            if not (self.lower <= float(field.data) <= self.upper):
                if self.hint:
                    raise ValidationError(self.hint)
                else:
                    raise ValidationError("Not in correct range")
```

As before, we can see it defines `__init__` and `__call__` and sets the expected answers in `__init__`, and then checks that `field.data` contains a valid response to the code.

There's two things worth noting here -

1. `CheckNumberRange()` takes the optional argument `hint`, meaning if we want to give the user an additional hint if they get an incorrect answer we can.
2. The `__call__` method doesn't actually return anything if it validates correctly, which is different to how we dealt with this scenario in `CorrectAnswer`. Both ways are fine, as only incorrect answers will result in raising a `ValidationError`. It's up to you how you want to structure the code to validate the inputs.

## Step 3: Making the quiz accessable

**Add a submission object in models.py**

Now we have a form that will display the questions and retrieve the input from the user, we need to create a submission object that will take the input and translate it into the appropriate form in order to be stored in and retrieved from the database.

These are created within the `models.py` module, but to keep this example a bit more self-contained lets create a new file `models_example.py` that we'll work in.

In `models_example.py` add the following to the top of the file

```python
from flask_sqlalchemy import SQLAlchemy
db = SQLAlchemy()
```

This is the connection to the database from Flask.

Then we can add -

```python
class SubmissionExampleQuiz(db.Model):

    __tablename__ = "example"

    submissionid = db.Column(db.Integer, primary_key=True)
    studentno = db.Column(db.Integer)
    submissiontime = db.Column(db.DateTime(timezone=True))
    correct = db.Column(db.Boolean)
    incomplete = db.Column(db.Boolean)

    q1 = db.Column(db.String())
    q2 = db.Column(db.String())
    q3_code = db.Column(db.LargeBinary)
    q4_image = db.Column(db.String())
    
    def __init__(self, studentno, submissiontime, correct, incomplete, q1, q2, q3_code, q4_image):
        self.studentno = studentno
        self.submissiontime = submissiontime
        self.correct = correct
        self.incomplete = incomplete
        self.q1 = q1
        self.q2 = q2
        self.q3_code = q3_code
        self.q4_image = q4_image
```

Lets have a closer look at this code -

```python
    __tablename__ = "example"
```

This is going to be the table we are going to submit the answers into, so it can be named anything (we just have to make sure it's unique and that when we create the table in the database we name it the same thing.

```python
    submissionid = db.Column(db.Integer, primary_key=True)
    studentno = db.Column(db.Integer)
    submissiontime = db.Column(db.DateTime(timezone=True))
    correct = db.Column(db.Boolean)
    incomplete = db.Column(db.Boolean)
```

These fields are exactly the same for any submission object we create. They are -

1. `submissionid` - This will be the unique, primary key for the database entry
2. `studentno` - The student number of the user submitting the form to be placed in the database
3. `submissiontime` - The time it was submitted (in order to track late submissions)
4. `correct` - A boolean stating whether the entire quiz was validated as correct
5. `incomplete` - A boolean stating whether the quiz was missing answers

From here we just add the questions that are applicable to that particular quiz. For most types of questions we create a column in the database with the String datatype - 
```python
db.Column(db.String())
```

With the exception of code, which is stored as a large binary file within the database -
```python
db.Column(db.LargeBinary())
```

Note that Coder Quiz stores images as Strings as well. This is because Coder Quiz uploads the images to a certain location on the server, then just stores the reference to this location in the database, and doesn't store the actual image in the database.

We then just add the `__init__` method, that takes each of the inputs and assigns them to a class variable of the same name.

```python
def __init__(self, studentno, submissiontime, correct, incomplete, q1, q2, q3_code, q4_image):
    self.studentno = studentno
    self.submissiontime = submissiontime
    self.correct = correct
    self.incomplete = incomplete
    self.q1 = q1
    self.q2 = q2
    self.q3_code = q3_code
    self.q4_image = q4_image
```

**Add a route in routes.py**
Now we need to add some logic that will tell the Flask application what to do if someone tries to access the page with our example quiz on it.

In `routes.py` find the following bit of code -

```python
@application.route(local("/biol3014inclass2"), methods=["GET", "POST"])
def biol3014_inclass2():
    form = BIOL3014InClass2()
    return create_page(form, 'biol3014inclass2.html')
```

And add below it a new decorator and function for our example quiz

```python
@application.route(local("/example_quiz"), methods=["GET", "POST"])
def example_quiz():
    form = ExampleQuiz()
    return create_page(form, 'example_quiz.html')
```  

We need to make sure that whatever we set form to is the same object we created in `forms_example.py`, and also we'll need to import it at the top of `routes.py` now as well.

So also add in 

```python
from forms_example import ExampleQuiz
```
To the top of `routes.py`

Most of the logic is actually handled in the `create_page` function, and we'll go into exactly how that works in another guide. For now, we can just accept that Coder Quiz will handle making our new page for us and that this is all we need to do to add a route.

**Create the default HTML page**

We do need to create the actual HTML page we directed the route to in `routes.py` - that is, we need to create `example_quiz.html`

In the `templates` folder, create a new HTML page called `example_quiz.html` - If you're happy to just use the default settings then Coder Quiz is also already setup to handle all of this automatically. Just copy the code from an existing quiz HTML file - say, `scie2100practical1.html` and paste it into `example_quiz.html`

The syntax in these files is a templating language called Jinja2 - http://jinja.pocoo.org/ . It allows us to use variables and logic such as for loops and if statements to generate HTML templates. This is why we can use the same code to generate different quizes. In the next section we'll look at how we can customise this HTML / CSS if we'd like.

**Add in links to the submission, query, and marking forms**

In order to check on their own submissions, users on Coder Quiz can use a submission form. Tutors can also use a query form, to check on anyone's submission. And finally, tutors can use a marking form to get a summary of all users for a certain quiz / form.

To make our new quiz accessible from all of these forms, we need to just add the Submission object we created in early (the form that we added to the `models_example.py` file.

We just need to add in a tuple containing the name of this object - in this case `SubmissionExampleQuiz` and the text we want to display to the user / tutor when they interact with these forms.

So, add the following

```python
('SubmissionExampleQuiz', 'Example Quiz')
```
to the `choices` list in the `assessment_item` variable in the `QueryForm`, `MarkingForm`, and `SubmissionForm` objects in `forms.py`. So you'll need to add that tuple in three times in this file.

**Add in a due date to due_dates.py**

In order to set a due date for when this submission should be completed - we need to add a due date in to `due_date.py`. Coder Quiz still accepts submissions after this date, it just stylises them to display <span style="color:red"> This submission was late</span> if anyone is viewing the submission.

Just add a due date with the appropriate date and time to the dictionary `duedates` in `due_dates.py`. Just set a date for off in the future for our Example Quiz.

```python
"ExampleQuiz": datetime.strptime("18/3/30 20:24", "%d/%m/%y %H:%M"),
```

**Add in a link to the new quiz from the landing page**

Now lets add in a link so that we can easily access the new quiz. In `templates/landing.html` we have the Jinja2 template that sets up the HTML for our landing page that contains all the available quizes.

Find the following code in `landing.html`

```python
    <a href="{{ url_for('practice') }}">Practice Quiz</a><br>
```
This adds in a link using url_for which will trigger the bit of logic we created for the route in `routes.py`
We need to make sure that whatever we pass into `url_for` matches what we added to the route -

```python
@application.route(local("/example_quiz"), methods=["GET", "POST"])
```
So, in this case we need to add in `example_quiz`

Add this link in `landing.html` now, just under the link for the `practice` quiz -
```python
    <a href="{{ url_for('example_quiz') }}">Practice Quiz</a><br>
```

**Viewing our progress so far**

These are all the steps we need to make a new quiz on Coder Quiz, so if we run `routes.py` now we should be able to navigate to the landing page, click on our quiz, and see the questions and the quiz appear. We will also see it appear as an option on the drop down menus if we follow the links `View Submissions`, `Query a single student's submission`, or `Generate a marking report for all students`.

We can validate the quiz by entering answers and clicking `Check Answers`, but we can't submit the quiz yet (because we'll need to create a table in the database for it to be submitted into and we'll do this in **Step 5**).

<span style="color:red"> Important note: We deliberatly created our forms and models in separate files so we could see exactly what we needed to import and how to make these files simply, but we really need the `SubmissionExampleQuiz` class present in the `models.py` module to work with the database. So, before we move on lets copy that class into `models.py`. No need to copy the import statements as they'll be already there.</span>




## Step 4: Custom styling to the template form
In **Step 3** we saw how we can use the default Jinja 2 template and have Coder Quiz automatically create the HTML for our page. If we look at SCIE2100 Practical 6, both as the raw `scie2100practical6.html` page and by navigating to SCIE2100 Practical 6 from a running version of Coder Quiz, we can see an example of how we can layout the individual pages explicitly, by placing the form item and the form item's label in the Jinja2 template.

A small example of how you could create a specific layout for our Example Quiz follows. Copy the following code into a new HTML file in the `templates` folder called `example_quiz_custom.html`.

```python

{% extends "layout.html" %}

{% block content %}
  <main class="hero-section">
    <div class="style-example">


      <div class="section-map">
        <div class="nav">

            Here is some new text to introduce the Example Quiz

            <p class = "big-error-message">{{ error }}</p>

          <form method="POST" enctype = "multipart/form-data" >
            {{ form.hidden_tag() }}

            <div class="form-group">

            {% with messages = get_flashed_messages() %}
                  {% if messages %}
                    <ul>
                    {% for message in messages %}
                        <li>{{ message }} </li>
                    {% endfor %}
                    </ul>
                  {% endif %}
                {% endwith %}
            </div>


                <div class="form-group">
            {{ form.q1.label }}
            {{ form.q1() }}

              {% if form.q1.errors %}
                {% for error in form.q1.errors %}
                  <p class="error-message">{{ error }}</p>
                {% endfor %}
              {% endif %}

            </div>

            <div class="form-group">
            {{ form.q2.label }}
            {{ form.q2() }}

              {% if form.q2.errors %}
                {% for error in form.q2.errors %}
                  <p class="error-message">{{ error }}</p>
                {% endfor %}
                {% elif form.check.data  %}
			{% if form.q2.data is not none %}
                		<p class="correct-message"> Accepted </p>
             		{% endif %}
		 {% endif %}

            </div>

            <div class="form-group">
            {{ form.q3_code.label }}
            {{ form.q3_code() }}

              {% if form.q3_code.errors %}
                {% for error in form.q3_code.errors %}
                  <p class="error-message">{{ error }}</p>
                {% endfor %}
                {% elif form.check.data %}
                {% if form.q3_code.data is not none %}
                <p class="correct-message"> Accepted </p>
              {% endif %}
              {% endif %}

            </div>

                  <div class="form-group">
                      <form-title>We can add a title here</form-title>

            {{ form.q4_image.label }}
            {{ form.q4_image() }}

                      We've removed the error messages for question four.

            </div>

            <div class>
            {{ form.check (class ="btn-primary")  }}
            {{ form.submit (class = "btn-primary")  }}
            </div>
             </form>
        </div>
        </div>
      <div class="clearfix"></div>
    </div>
  </main>


{% endblock %}


```

What are we doing differently here? We change

```python
    <div class="container">
```

To 
```python
    <div class="style-example">
```

So we can create some custom CSS for just this form, and we add in some titles and text to the form and deliberatly hide the error messages from Question Four. Importantly, we need to specifically add in any questions and the label for the question to the HTML if we want to display it to the user.

```python
            {{ form.q1.label }}
            {{ form.q1() }}
```

Is the Jinja 2 syntax for doing this. How does it know which form to use? Because in `routes.py` we specified which form is associated with each page, and passed it in directly to the page when rendering it.


It's easiest to see these changes by rendering them, so let's do that.

**Point the route to the new HTML file**

We need to change the URL endpoint that `routes.py` is trying to access - so change

```python
@application.route(local("/example_quiz"), methods=["GET", "POST"])
def example_quiz():
    form = ExampleQuiz()
    return create_page(form, 'example_quiz.html')
```  

To -

```python
@application.route(local("/example_quiz"), methods=["GET", "POST"])
def example_quiz():
    form = ExampleQuiz()
    return create_page(form, 'example_quiz_custom.html')
```  
**Create some custom CSS styles**

And we need to add in some custom CSS to the `main.css` file, sitting in the `static/css` folder.

Try adding in this CSS to the top of `main.css`

```css
.style-example{
  background: whitesmoke;
  font-family: 'Open Sans', sans-serif;
  float: left;
  padding: 20px;
  color: mediumpurple;

}
```

Now if we re-run the Flask application and navigate to the Example Quiz page, you should see the changes such as the form being pulled to the left and the purple colouring being used to stylise the new headings and text.

This is a simple example, but really anything you can do in HTML or CSS can be applied in the same manner in order to give you control over how you present the forms to the user.


## Step 5: Making a table in the database

Creating a new table in the database is fairly straightforward. Essentialy we need to create a table in the database that mirrors the names and information we have in the `SubmissionExampleQuiz` class we created in `models_example.py`.

Connect to postgres on the command line via

```
sudo -u postgres psql
```

Then copy and paste the following into the command line

``` 
CREATE TABLE example (
    submissionid SERIAL PRIMARY KEY NOT NULL,
    studentno INTEGER,
    submissiontime TIMESTAMP,
    correct BOOLEAN,
    incomplete BOOLEAN,
    q1 TEXT NOT NULL,
    q2 TEXT NOT NULL,
    q3_code BYTEA NOT NULL,
    q4_image TEXT NOT NULL
);
```

The syntax here is pretty easy to understand. After `CREATE TABLE` make sure that the name of the table matches the name you supplied in the `__tablename__` variable in the `SubmissionExampleQuiz` class in `models_example.py` Then the next five lines are always the same (because every submission needs an id, student number, time, correct boolean, and incomplete boolean).

Finally, you need a field for each question in your quiz, and either give it a value of `TEXT NOT NULL` or if it is storing code `BYTEA NOT NULL`. Separate each field with a comma, don't place a comma after the last field. 

Finally, because we're serving this up on the web on the server, the user `www-data` needs to be able to access the database, so grant all privileges to this table to that user. You can grant `www-data` access to all tables and sequences by entering the two following commands into the psql command line -

```
grant all privileges on all tables in schema public  to "www-data";
grant all privileges on all sequences in schema public  to "www-data";
```

Now we should be able to store our quiz in the database and retrieve the results as well.

## Checklist of steps for creating new form

1. Create a new form (and create any custom validators you need)
2. Add the route handling to routes.py
3. Create a new HTML page with the default template (or a custom template)
4. Add in a link on the submission, query, and marking forms in forms.py
5. Add in a due date to due_dates.py
6. Create a table in the database
7. Check that it submits correctly into the database and that answers can be retrieved